In [50]:
import os

In [51]:
%pwd

'd:\\Machine Learning\\Wine-Classification'

In [52]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
      root_dir: Path
      url: str
      local_data_file: Path
      unzip_file: Path

In [ ]:
from src.wineModel.utils import *
from src.wineModel.constants import *


class ConfigurationManager:
      def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
      ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directory([self.config.artifacts_root])
      
      def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directory([config.root_dir])
            
            data_ingestion_config = DataIngestionConfig(
                  root_dir=config.root_dir,
                  url = config.source_url,
                  local_data_file = config.local_data_file,
                  unzip_file=config.unzip_dir
            )
            return data_ingestion_config
      

In [54]:
import os
import zipfile
import urllib.request as r
from src.wineModel import logger


class DataIngestion:
      def __init__(self, config: DataIngestionConfig):
            self.config = config

      def download_file(self):
            if not os.path.exists(self.config.local_data_file):
                  filename, header = r.urlretrieve(
                        url = self.config.url,
                        filename = self.config.local_data_file
                  )
                  logger.info(f"{filename} download with following info: \n{header}")
            else:
                  logger.info("File is already exist")
      
      
      def extract_zip_file(self):
            unzip_file = self.config.unzip_file
            os.makedirs(unzip_file, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as f:
                  f.extractall(unzip_file)

In [55]:
try:
      config = ConfigurationManager()
      data_ingestion = config.get_data_ingestion_config()
      data = DataIngestion(config=data_ingestion)
      data.download_file()
      data.extract_zip_file()
except Exception as e:
      raise e

[2026-02-12 15:36:24,660: INFO: __init__: yaml file: config\config.yaml is loaded successfully]
[2026-02-12 15:36:24,662: INFO: __init__: yaml file: params.yaml is loaded successfully]
[2026-02-12 15:36:24,669: INFO: __init__: yaml file: schema.yaml is loaded successfully]
[2026-02-12 15:36:24,672: INFO: __init__: Directory is created at: artifacts]
[2026-02-12 15:36:24,673: INFO: __init__: Directory is created at: artifacts/data_ingestion]
[2026-02-12 15:36:25,764: INFO: 3418529093: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 21293
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "40c416adcb18016196e44b68d80ad9815af11f96ec0129f98beb2bacb85bc434"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E25C:1ED50C:8577B3:13799CA:698D9E18
Accept-Ra